# Chat Testing

Here I am going to be figuring out all of the different aspects of how the chatbot will work.

In [1]:
from openai import OpenAI
import json
import pg8000

In [2]:
# Load Chat API Key and DataBase Data
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv(dotenv_path="./.env")

# Access the OPENAI_API_KEY variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

DATABASE_URL = os.getenv("POSTGRES_URL")
DATABASE_USER = os.getenv("POSTGRES_USER")
DATABASE_PASS = os.getenv("POSTGRES_PASSWORD")
DATABASE_DB = os.getenv("POSTGRES_DATABASE")
DATABASE_HOST = os.getenv("POSTGRES_HOST")


### Basic Load of OpenAI

In [43]:
client = OpenAI(api_key=OPENAI_API_KEY)

try:
  completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {
              "role": "user",
              "content": "Write a haiku about recursion in programming."
          }
      ]
  )

  print(completion.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {e}")

Function calls itself,  
Layers deep in endless swirl,  
Return brings us home.


### Differentiate between Recipe, General, or Lifestyle

In [44]:
user_question = "Why do you say that?"

user_diet = "celiac diet and low fodmap diet"

rules = """
1. You are the first step in a longer chatbot which has the goal of helping users
with specific dietary needs to answer general questions about the dietary needs, provide lifestyle advice 
in the context of said dietary need, or provide recipes specific to the user's needs.
2. Your job is only to differentiate the requests, so the next request to the chatbot will be able to have
a specific format.
For example, recipes will be returned in a different format from lifestyle advice, which will be
different from general queries.
3. Your job is to differentiate between general queries, lifestyle tips, and recipe requests.
4. If someone is asking for a recipe, you will respond with only 'recipe'.
5. If someone is asking general questions about their dietary needs, you will answer with "general"
6. If someone is asking about things that will be useful general knowledge living with their dietary need, you will answer "lifestyle"
7. Anything the user asks which does not pertain their diet, a diet, or the lifestyle associated with that diet, or claryfying questions, answer "not applicable"
"""


try:
  completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[
          {"role": "system", "content": f"{rules}"},
          {"role": "assistant", "content": f"{user_diet}"},
          {
              "role": "user",
              "content": f"{user_question}"
          }
      ]
  )

  print(completion.choices[0].message.content)
  query = completion.choices[0].message.content
except Exception as e:
    print(f"An error occurred: {e}")


not applicable


### Recipe Responses

In [ ]:
# Create the chat completion request for recipes
response = client.chat.completions.create(
    model="gpt-4o-mini",  
    messages=[
        {
            "role": "system",
            "content": """You give recipes given certain dietary conditions. 
            You must make sure the recipe is not repeated.
            You must make sure the recipe does not recommend anything the user cannot eat
            with their dietary restrictions"""
        },
        {
            "role": "assistant",
            "content": "Dietary restrictions: User has the following di"
        },
        {
            "role": "user",
            "content": "I would like a basic recipe with chicken and vegetables which takes less than 30 minutes."
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recipe_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "Recipe": {
                        "description": "The name of the recipe",
                        "type": "string"
                    },
                    "Ingredients": {
                        "description": "list of ingredients with measurements if needed",
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                    "Prep Time": {
                        "description": "time in minutes to prepare this recipe",
                        "type": "integer"
                    },
                    "Instructions": {
                    "description": "Instructions to create this recipe, stored as JSON",
                    "type": "array",
                    "items": {
                        "type": "string"
                    }
                },
                "additionalProperties": False
            }
        }
    }
)

# Print the response
print(response.choices[0].message.content)
recipe = json.loads(response.choices[0].message.content)

{"Recipe":"Quick Chicken and Vegetable Stir-Fry","Ingredients":["1 lb chicken breast, sliced","2 cups mixed vegetables (bell peppers, broccoli, carrots)","2 tablespoons olive oil","2 tablespoons soy sauce (gluten-free)","1 teaspoon garlic, minced","Salt and pepper to taste"],"Prep Time":15,"Instructions":"1. Heat the olive oil in a large skillet over medium-high heat. 2. Add the sliced chicken breast and cook for about 5-7 minutes, or until no longer pink. 3. Stir in the minced garlic and cook for another minute. 4. Add the mixed vegetables to the skillet and stir-fry for about 5-7 minutes until they are tender-crisp. 5. Pour in the gluten-free soy sauce and stir to combine. Season with salt and pepper as desired. 6. Serve immediately."}


In [26]:
recipe['Recipe']

'Quick Chicken and Vegetable Stir-Fry'

In [4]:
conn = pg8000.connect(
    user=DATABASE_USER,
    password=DATABASE_PASS,
    host=DATABASE_HOST,
    database=DATABASE_DB
)
cursor = conn.cursor()

# Fetch table names from Database
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public';
""")
tables = cursor.fetchall()
print("Tables in the database:", [table[0] for table in tables])

# Fetch Column names from user table
cursor.execute("""
    SELECT column_name 
    FROM information_schema.columns 
    WHERE table_name = 'users';
""")
columns = cursor.fetchall()
print("Columns in 'users' table:", [col[0] for col in columns])

# Fetch Column names from user table
cursor.execute("""
    SELECT column_name 
    FROM information_schema.columns 
    WHERE table_name = 'userpreference';
""")
columns = cursor.fetchall()
print("Columns in 'userpreference' table:", [col[0] for col in columns])

# Fetch Column names from user table
cursor.execute("""
    SELECT column_name 
    FROM information_schema.columns 
    WHERE table_name = 'diettype';
""")
columns = cursor.fetchall()
print("Columns in 'diettype' table:", [col[0] for col in columns])

# Fetch Column names from user table
cursor.execute("""
    SELECT column_name 
    FROM information_schema.columns 
    WHERE table_name = 'context';
""")
columns = cursor.fetchall()
print("Columns in 'context' table:", [col[0] for col in columns])

# Fetch Column names from user table
cursor.execute("""
    SELECT column_name 
    FROM information_schema.columns 
    WHERE table_name = 'tip';
""")
columns = cursor.fetchall()
print("Columns in 'tip' table:", [col[0] for col in columns])

# Fetch Column names from user table
cursor.execute("""
    SELECT column_name 
    FROM information_schema.columns 
    WHERE table_name = 'recipe';
""")
columns = cursor.fetchall()
print("Columns in 'recipe' table:", [col[0] for col in columns])

# Example query: Fetch User table
cursor.execute("SELECT * FROM users LIMIT 5;")
rows = cursor.fetchall()
print(rows)



# Close the connection
conn.close()

# Return fetched context as a dictionary
# return {key: value for key, value in rows}

Tables in the database: ['diettype', 'tip', 'userpreference', 'context', 'recipe', 'users']
Columns in 'users' table: ['uid', 'email', 'password', 'username']
Columns in 'userpreference' table: ['preferenceId', 'uid', 'dietId']
Columns in 'diettype' table: ['dietId', 'dietName']
Columns in 'context' table: ['tipId', 'timestamp', 'uid', 'chatId', 'recipeId', 'chatResponse', 'userQuestion']
Columns in 'tip' table: ['tipId', 'uid', 'timestamp', 'tip', 'summary']
Columns in 'recipe' table: ['ingredients', 'uid', 'timestamp', 'recipeId', 'instructions', 'prepTime', 'recipeName']
([5, 'michael.moser@atlasschool.com', None, 'Michael Moser'], [6, 'emptympuzzle@gmail.com', None, 'Puzzle'], [7, 'puzzleemptym@gmail.com', None, 'PuzzleEmptyM'], [8, 'samuelclausgreetingcards@gmail.com', None, 'Samuel Moser'], [9, 'rachael.bradford@atlasschool.com', None, 'Rachael Bradford'])
